# AdaBoost Classifier

## Several instances of weak Decision Tree classifiers with each instance having the weightage tweaked to correct the pervious instance classifiers errors

### Import the necessary packages and fetch the data from SQL Server

In [1]:
import pyodbc 
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

### Fetch data from local sql server for training

In [2]:

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-HVRMUNPF;'
                      'PORT=1433;'
                      'Database=RENTERS_STP;'
                      'Trusted_Connection=yes;'
                      )

query = 'SELECT  * from [RENTERS_STP].[dbo].[DR_DetailedRequest_classification]'

    
df = pd.read_sql(query, conn)

### Create 'features' and 'labels' from the dataset. Split the data into training and test datasets

In [3]:
y=df['PREDICTION_VALUE_Y_BOOL']
X = pd.get_dummies(df.drop(['PREDICTION_VALUE_Y_BOOL'], axis=1)).fillna(0)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state=42, stratify=y)



### Instantiate the Decision tree and the Adaboost classifiers

In [4]:
dt = DecisionTreeClassifier(max_depth=2, random_state=1)

ada = AdaBoostClassifier(base_estimator=dt, n_estimators=180, random_state=1)

### Fit and train the model

In [5]:
ada.fit(X_train,y_train)

# Compute the probabilities of obtaining the positive class
y_pred_proba = ada.predict_proba(X_test)[:,1]


### Model Evaluation

In [6]:
ada_roc_auc = roc_auc_score(y_test, y_pred_proba)

print('ROC AUC score: {:.6f}'.format(ada_roc_auc))

ROC AUC score: 0.947582
